In [730]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

In [731]:
def preprocess(classes, features):
   # Read the dataset
   data = pd.read_csv('./dataset/dry_bean_dataset.csv')
   # Filter rows based on the 'Class' column
   # Perform linear interpolation for missing values in the 'MinorAxisLength' column
   data[features] = data[features].copy().fillna(data[features].mean())
   # Manually perform Min-Max scaling
   for column in features:
      min_val = data[column].min()
      max_val = data[column].max()
      data[column] = (data[column] - min_val) / (max_val - min_val)

   # Shuffle the data
   # data = data.sample(frac=1, random_state=0).reset_index(drop=True)
   data = shuffle(data, random_state=0)
   data.insert(0, 'bias', 1)
   features.append("bias")
   X = data[features].values
   Y = np.where(data['Class'] == classes[0], -1, np.where(data['Class']==classes[1],0,1))
   x_train, x_test, y_train, y_test = train_test_split(
      X, Y, test_size=0.4, random_state=42)
   return x_train, x_test, y_train, y_test

In [732]:
x_train,x_test,y_train,y_test = preprocess(["BOMBAY","CALI","SIRA"],["Area","Perimeter","MajorAxisLength","MinorAxisLength","roundnes"])

x_train.shape

(90, 6)

In [733]:
def sigmoidFunc(x):
    return (1/(1+np.exp(-x)))
def derivative_sigmoid(x):
    return x* (1-x)

In [734]:
def initialize_weights(hiddenlayers,n_neurons):
   weights = [np.random.rand(n_neurons[i], n_neurons[i+1]) for i in range(hiddenlayers+1)]
   return weights

In [735]:
weights = initialize_weights(2,[6,3,4,3])

In [736]:
# Calculate neuron activation for an input
def activate_input(weights, inputs):
    activation = 0
    for i in range(len(weights.T)):
        for j in range(len(weights)):
            activation += weights[j] * inputs[i][j]
    activation=sigmoidFunc(activation)
    return activation

In [737]:
def activate_hidden(weights, activatedInput):
    activation = 0
    for i in range(len(weights)):
        for j in range(len(weights)):
            activation += weights[j] * activatedInput[i]
    activation=sigmoidFunc(activation)
    return activation

In [738]:
activations=[]
newInp = activate_input(weights[0],x_train)
activations.append(newInp)
for i in range(1,len(weights)):
    newInp =activate_hidden(weights[i],newInp)
    activations.append(newInp)
activations

[array([0.99755226, 0.99816035, 0.99710343]),
 array([0.99092628, 0.99927955, 0.95135737, 0.98043746]),
 array([0.99978699, 0.99930183, 0.99999711])]

In [739]:
def backprop_out(y_train,activations):
    for i in y_train:
        output_layer_errors = activations[-1] - i
        delta = output_layer_errors * derivative_sigmoid(output_layer_errors)
    return delta

In [886]:
def backprop_hidden(delta,act):
    print(act)
    for j in range(len(act)-1):
        hidden_layer_error = np.dot(delta[j],act)
        hidden_delta = hidden_layer_error * derivative_sigmoid(hidden_layer_error)
    return hidden_delta

In [888]:
Deltas =[]
delta = backprop_out(y_train,activations)
Deltas.append(delta)
activation =activations[:len(activations)-1]
for i in activation[::-1]:
    delta =backprop_hidden(delta,i)
    Deltas.append(delta)
Deltas

[0.99092628 0.99927955 0.95135737 0.98043746]
0.9909262758739313 [2.12918252e-04 6.97199376e-04 2.88995042e-06]
0.9992795501582108 [2.12918252e-04 6.97199376e-04 2.88995042e-06]
0.9513573687437772 [2.12918252e-04 6.97199376e-04 2.88995042e-06]
[0.99755226 0.99816035 0.99710343]
0.9975522555332673 [8.20091345e-12 8.33975953e-12 7.55904554e-12 8.02822154e-12]
0.9981603488495935 [8.20091345e-12 8.33975953e-12 7.55904554e-12 8.02822154e-12]


[array([2.12918252e-04, 6.97199376e-04, 2.88995042e-06]),
 array([8.20091345e-12, 8.33975953e-12, 7.55904554e-12, 8.02822154e-12]),
 array([6.92115167e-23, 6.92959231e-23, 6.91492509e-23])]

In [933]:
def update_weights_out(weights, deltas, activations, learning_rate):
    new_weight = []
    deltas = deltas[::-1]
    for i in range(weights):
            for j in range(len(weights[i])):
                dotprod = np.dot(deltas[i][j],activations[i])
                mult = learning_rate * dotprod
                plus = weights[i][j]+mult
                new_weight.append(plus)
    return new_weight

In [934]:
weights_updated_out=[]
for i in range(len(weights)):
    weights_updated_out.append(update_weights_out(weights[i],Deltas,activations,0.01))

TypeError: only integer scalar arrays can be converted to a scalar index

[array([0.26660301, 0.26660785, 0.26660091]),
 array([0.71384877, 0.71385361, 0.71384667]),
 array([0.45727324, 0.45727808, 0.45727114]),
 array([0.65023137, 0.65023138, 0.65023137]),
 array([0.50266884, 0.50266884, 0.50266884]),
 array([0.43139357, 0.43139357, 0.43139357]),
 array([0.71776646, 0.71776646, 0.71776646]),
 array([0.01236084, 0.01236084, 0.01236084]),
 array([0.08029438, 0.08029438, 0.08029438])]

In [744]:
mse = ((output_layer_outputs - y_train)**2).sum() / (2*N)
mse

ValueError: operands could not be broadcast together with shapes (90,3) (90,) 

In [ ]:
# feedforward
hidden_layer_inputs = np.dot(x_train, weights_1)
hidden_layer_outputs = sigmoidFunc(hidden_layer_inputs)

output_layer_inputs = np.dot(hidden_layer_outputs, weights_2)
output_layer_outputs = sigmoidFunc(output_layer_inputs)

In [ ]:
mse = ((output_layer_outputs - y_train)**2).sum() / (2*N)
mse

0.4123640602881693

In [ ]:
calculate_accuracy(x_test,y_test,[weights_1,weights_2])

NameError: name 'predict' is not defined

In [ ]:
def calculate_accuracy(X_test, y_test,trained_weights):
   predictions =predict(X_test,trained_weights)
   predictions = predictions.flatten()
   correct_predictions = sum(predictions == y_test)
   accuracy = correct_predictions / len(y_test)
   return accuracy

In [ ]:
def predict(inputs, trained_weights):
   outputs = forward_step(inputs, trained_weights)
   return outputs[-1]